<Header> This is is where we develop a baseline factor model that allows us to take certain factors and use them in multi-OLS regression to predict our excess return signals on a security level. These signals are absolute. </Header>

In [1]:
#First, import our packages for the database connection and dataframe access
import mysql.connector
import pandas as pd
import statsmodels.api as sm
import numpy as np
import warnings
#pd datareader not working
#import pandas_datareader as pdr

In [2]:
warnings.filterwarnings("ignore")

In [3]:
# Connect to MySQL database
connection = mysql.connector.connect(
    host="ubctg.con7266gcvin.us-east-2.rds.amazonaws.com",
    user="admin",
    password="ubctgquant",
    database="ubctg"
)

Here, we pull monthly returns across the stock universe over 10 years from 2011 to 2021

In [4]:
# Create a cursor object to execute SQL queries
cursor = connection.cursor()

# Define the start and end dates
start_date = '2011-01-01'
end_date = '2021-01-31'

# SQL query to retrieve data from the "Volatility" table between two dates
sql_query = f"SELECT * FROM `Monthly Returns` mr WHERE Date BETWEEN '{start_date}' AND '{end_date}'"

# Execute the SQL query
cursor.execute(sql_query)

# Fetch all rows from the result set
universe_data = cursor.fetchall()

# Convert fetched data into a pandas DataFrame
columns = [i[0] for i in cursor.description]  # Extract column names from cursor description

#create new df
universe_df = pd.DataFrame(universe_data, columns=columns)

#close cursor and db connection
cursor.close()

# Print the DataFrame
universe_df

,PERMNO,date,COMNAM,NAICS,PERMCO,DCLRDT,DLPDT,PAYDT,RCRDDT,FACPR,FACSHR,ACPERM,ACCOMP,DLPRC,PRC,VOL,RET,BID,ASK
0,10001,2011-01-31,GAS NATURAL INC,221210,7953,2011-01-05,,2011-01-31,2011-01-14,0.0,0.0,,,NaN,10.78000,4648.0,0.028992,10.72000,10.74000
1,10001,2011-02-28,GAS NATURAL INC,221210,7953,2011-01-26,,2011-02-28,2011-02-15,0.0,0.0,,,NaN,10.98000,4586.0,0.022727,10.96000,10.99000
2,10001,2011-03-31,GAS NATURAL INC,221210,7953,2011-02-23,,2011-03-31,2011-03-15,0.0,0.0,,,NaN,11.73000,4137.0,0.072404,11.72000,11.73000
3,10001,2011-04-29,GAS NATURAL INC,221210,7953,2011-03-30,,2011-04-29,2011-04-15,0.0,0.0,,,NaN,11.23000,4109.0,-0.038789,11.21000,11.23000
4,10001,2011-05-31,GAS NATURAL INC,221210,7953,2011-04-27,,2011-05-31,2011-05-13,0.0,0.0,,,NaN,11.50000,4979.0,0.028050,11.46000,11.50000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887531,93436,2020-09-30,TESLA INC,336111,53453,,,,,NaN,NaN,,,NaN,429.01001,17331954.0,-0.139087,428.82001,429.01999
887532,93436,2020-10-30,TESLA INC,336111,53453,,,,,NaN,NaN,,,NaN,388.04001,8330610.0,-0.095499,388.04001,388.35001
887533,93436,2020-11-30,TESLA INC,336111,53453,,,,,NaN,NaN,,,NaN,567.59998,7811501.0,0.462736,566.09998,566.85999
887534,93436,2020-12-31,TESLA INC,336111,53453,,,,,NaN,NaN,,,NaN,705.66998,11962716.0,0.243252,705.21002,705.50000


Generate the number of observations per security in our table. This way we can remove those that do not have a full dataset

In [5]:
# Create a cursor object to execute SQL queries
cursor = connection.cursor()

# Define the start and end dates
start_date = '2011-01-01'
end_date = '2021-01-31'

# SQL query to retrieve data from the "Volatility" table between two dates
sql_query = f"SELECT COUNT(PERMNO), PERMNO FROM `Monthly Returns` mr WHERE Date BETWEEN '{start_date}' AND '{end_date}' GROUP BY PERMNO"

# Execute the SQL query
cursor.execute(sql_query)

# Fetch all rows from the result set
observationTable_data = cursor.fetchall()

# Convert fetched data into a pandas DataFrame
columns = [i[0] for i in cursor.description]  # Extract column names from cursor description

#create new df
observation_df = pd.DataFrame(observationTable_data, columns=columns)

#close cursor and db connection
cursor.close()

# Print the DataFrame
observation_df

unedited_observation_df = observation_df

Here, we remove securities that do not have enough observations (in our case, we look for at least 120)

In [6]:
#Filter securities with at least 120 observations in the period
observation_df_filtered = observation_df[pd.to_numeric(observation_df['COUNT(PERMNO)']) >=120]

#Inner join our dataframes to only keep the securities we have data on
universe_df_filtered = pd.merge(universe_df, observation_df_filtered, on='PERMNO', how='inner')

universe_df_filtered

,PERMNO,date,COMNAM,NAICS,PERMCO,DCLRDT,DLPDT,PAYDT,RCRDDT,FACPR,FACSHR,ACPERM,ACCOMP,DLPRC,PRC,VOL,RET,BID,ASK,COUNT(PERMNO)
0,10026,2011-01-31,J & J SNACK FOODS CORP,311821,7976,,,,,NaN,NaN,,,NaN,42.47000,12928.0,-0.119610,42.47000,42.55000,121
1,10026,2011-02-28,J & J SNACK FOODS CORP,311821,7976,,,,,NaN,NaN,,,NaN,43.97000,13651.0,0.035319,43.97000,43.99000,121
2,10026,2011-03-31,J & J SNACK FOODS CORP,311821,7976,2011-02-09,,2011-04-06,2011-03-15,0.0,0.0,,,NaN,47.07000,14227.0,0.073175,46.96000,47.08000,121
3,10026,2011-04-29,J & J SNACK FOODS CORP,311821,7976,,,,,NaN,NaN,,,NaN,50.82000,8807.0,0.079669,50.74000,50.82000,121
4,10026,2011-05-31,J & J SNACK FOODS CORP,311821,7976,,,,,NaN,NaN,,,NaN,51.52000,9658.0,0.013774,51.50000,51.52000,121
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478703,93436,2020-09-30,TESLA INC,336111,53453,,,,,NaN,NaN,,,NaN,429.01001,17331954.0,-0.139087,428.82001,429.01999,121
478704,93436,2020-10-30,TESLA INC,336111,53453,,,,,NaN,NaN,,,NaN,388.04001,8330610.0,-0.095499,388.04001,388.35001,121
478705,93436,2020-11-30,TESLA INC,336111,53453,,,,,NaN,NaN,,,NaN,567.59998,7811501.0,0.462736,566.09998,566.85999,121
478706,93436,2020-12-31,TESLA INC,336111,53453,,,,,NaN,NaN,,,NaN,705.66998,11962716.0,0.243252,705.21002,705.50000,121


We can now add a few factors to our model, in this case we will use GDP, CPI (inflation data), and the unemployment rate in each month. We then add it to our monthly returns dataframe for our OLS regression

In [7]:
#Introduce our factor data below as a CSV (pandas datareader not currently working for FRED api)
macro_factors = pd.read_csv("UBCTG Factor Model Example - Macro FRED Data.csv")

#these dates are at the beginning of month, so we will operate on our monthly return dataframe to convert our dates to the beginning of the month
universe_df_filtered["date"] = pd.to_datetime(universe_df_filtered["date"]).dt.to_period('M').dt.to_timestamp()
#display(universe_df_filtered)

macro_factors.dtypes
macro_factors["date"] = pd.to_datetime(macro_factors["DATE"])
macro_factors = macro_factors.drop('DATE', axis=1)
#display(macro_factors)

universe_df_with_external_factors = pd.merge(universe_df_filtered, macro_factors, on= 'date', how='inner')

#display(universe_df_with_external_factors)

universe_df_with_external_factors[universe_df_with_external_factors.index.isin([545])]

#universe_df_with_external_factors.loc[universe_df_with_external_factors["RET"] == "I"]
universe_df_with_external_factors["RET"] = pd.to_numeric(universe_df_with_external_factors["RET"], errors="coerce")

#To do: Lag and fractionally differentiate the data
#universe_df_with_external_factors["Ret_lag"] = universe_df_with_external_factors["RET"].shift(-1)
display(universe_df_with_external_factors)

,PERMNO,date,COMNAM,NAICS,PERMCO,DCLRDT,DLPDT,PAYDT,RCRDDT,FACPR,...,DLPRC,PRC,VOL,RET,BID,ASK,COUNT(PERMNO),UNRATE,CPI (USACPALTT01CTGYM),"Annualized Percent Change of GDP from Preceding Period, Seasonally Adjusted"
0,10026,2011-01-01,J & J SNACK FOODS CORP,311821,7976,,,,,NaN,...,NaN,42.47000,12928.0,-0.119610,42.47000,42.55000,121,9.1,0.168134,-3.034413
1,10026,2011-02-01,J & J SNACK FOODS CORP,311821,7976,,,,,NaN,...,NaN,43.97000,13651.0,0.035319,43.97000,43.99000,121,9.0,0.223168,-1.354698
2,10026,2011-03-01,J & J SNACK FOODS CORP,311821,7976,2011-02-09,,2011-04-06,2011-03-15,0.0,...,NaN,47.07000,14227.0,0.073175,46.96000,47.08000,121,9.0,0.283405,4.315752
3,10026,2011-04-01,J & J SNACK FOODS CORP,311821,7976,,,,,NaN,...,NaN,50.82000,8807.0,0.079669,50.74000,50.82000,121,9.1,0.307888,4.146784
4,10026,2011-05-01,J & J SNACK FOODS CORP,311821,7976,,,,,NaN,...,NaN,51.52000,9658.0,0.013774,51.50000,51.52000,121,9.0,0.346291,2.485213
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478703,93436,2020-09-01,TESLA INC,336111,53453,,,,,NaN,...,NaN,429.01001,17331954.0,-0.139087,428.82001,429.01999,121,7.8,1.332004,10.259221
478704,93436,2020-10-01,TESLA INC,336111,53453,,,,,NaN,...,NaN,388.04001,8330610.0,-0.095499,388.04001,388.35001,121,6.8,1.130982,-3.677010
478705,93436,2020-11-01,TESLA INC,336111,53453,,,,,NaN,...,NaN,567.59998,7811501.0,0.462736,566.09998,566.85999,121,6.7,1.122155,-4.401027
478706,93436,2020-12-01,TESLA INC,336111,53453,,,,,NaN,...,NaN,705.66998,11962716.0,0.243252,705.21002,705.50000,121,6.7,1.323012,3.579485


In [8]:
#columns_to_regress = ['Annualized Percent Change of GDP from Preceding Period, Seasonally Adjusted','CPI (USACPALTT01CTGYM)','UNRATE']

#Explore fractional differentiation of ask/bid or adjusted price to maintain trend in the data
new_observation_df = unedited_observation_df
uniqueTickerList = new_observation_df[pd.to_numeric(new_observation_df['COUNT(PERMNO)']) >=120]

#Unique tickers
uniqueTickerList1 = uniqueTickerList['PERMNO']
universe_df_with_external_factors.dtypes


PERMNO                                                                                  int64
date                                                                           datetime64[ns]
COMNAM                                                                                 object
NAICS                                                                                  object
PERMCO                                                                                  int64
DCLRDT                                                                                 object
DLPDT                                                                                  object
PAYDT                                                                                  object
RCRDDT                                                                                 object
FACPR                                                                                 float64
FACSHR                                                      

In [9]:
#Initialize large df to drop results of regression for each security at each regression date
containerdf = pd.DataFrame()
LookBack_Window=48

for ticker in uniqueTickerList1:
    ticker_specific_universe_df = universe_df_with_external_factors.loc[universe_df_with_external_factors['PERMNO'] == ticker]

    ticker_specific_universe_df["B0"] = 0
    ticker_specific_universe_df["R_squared"] = 0
    ticker_specific_universe_df["Beta_CPI"] = 0
    ticker_specific_universe_df["Beta_GDP"] = 0
    ticker_specific_universe_df["Beta_UnemRate"] = 0
    ticker_specific_universe_df["p_value_Alpha"] = 0
    ticker_specific_universe_df["p_value_Market"] = 0
    ticker_specific_universe_df["p_value_GDP"] = 0
    ticker_specific_universe_df["p_value_UnemRate"] = 0
    ticker_specific_universe_df= ticker_specific_universe_df.drop(["FACSHR","ACPERM","ACCOMP","DLPRC","BID","ASK","COUNT(PERMNO)","DCLRDT","DLPDT","PAYDT","RCRDDT","FACPR","FACSHR","ACPERM","ACCOMP","DLPRC"],axis=1)
    #for each 48-month (2-year) span, train an OLS and collect the results 
    for x in range(0, len(ticker_specific_universe_df)-LookBack_Window-1):
        # Define the independent variables (X) and dependent variable (y)
        X = ticker_specific_universe_df[["Annualized Percent Change of GDP from Preceding Period, Seasonally Adjusted","CPI (USACPALTT01CTGYM)","UNRATE"]][x:x+LookBack_Window]
        Y = ticker_specific_universe_df['RET'][x:x+LookBack_Window]

        # Add a constant term to the independent variables, check impact
        X = sm.add_constant(X)
    
        # Fit the linear regression model
        model = sm.OLS(Y, X)
        results = model.fit()
    
        # Get the coefficients
        coefficients = results.params
        #print(coefficients)
        # Get the coefficients
        ticker_specific_universe_df.loc[ticker_specific_universe_df.index[x+1+LookBack_Window], 'B0'] = results.params[0]
        ticker_specific_universe_df.loc[ticker_specific_universe_df.index[x+1+LookBack_Window], 'Beta_GDP'] = results.params[1]
        ticker_specific_universe_df.loc[ticker_specific_universe_df.index[x+1+LookBack_Window], 'Beta_CPI'] = results.params[2]
        ticker_specific_universe_df.loc[ticker_specific_universe_df.index[x+1+LookBack_Window], 'Beta_UnemRate'] = results.params[3]
        ticker_specific_universe_df.loc[ticker_specific_universe_df.index[x+1+LookBack_Window], 'R_squared'] = results.rsquared
        ticker_specific_universe_df.loc[ticker_specific_universe_df.index[x+1+LookBack_Window], 'p_value_B0'] = results.pvalues[0]
        ticker_specific_universe_df.loc[ticker_specific_universe_df.index[x+1+LookBack_Window], 'p_value_GDP'] = results.pvalues[1]
        ticker_specific_universe_df.loc[ticker_specific_universe_df.index[x+1+LookBack_Window], 'p_value_CPI'] = results.pvalues[2]
        ticker_specific_universe_df.loc[ticker_specific_universe_df.index[x+1+LookBack_Window], 'p_value_UnemRate'] = results.pvalues[3]
        
    display(ticker_specific_universe_df)
    break

,PERMNO,date,COMNAM,NAICS,PERMCO,PRC,VOL,RET,UNRATE,CPI (USACPALTT01CTGYM),...,R_squared,Beta_CPI,Beta_GDP,Beta_UnemRate,p_value_Alpha,p_value_Market,p_value_GDP,p_value_UnemRate,p_value_B0,p_value_CPI
0,10026,2011-01-01,J & J SNACK FOODS CORP,311821,7976,42.47000,12928.0,-0.119610,9.1,0.168134,...,0.000000,0.000000,0.000000,0.000000,0,0,0.000000,0.000000,NaN,NaN
1,10026,2011-02-01,J & J SNACK FOODS CORP,311821,7976,43.97000,13651.0,0.035319,9.0,0.223168,...,0.000000,0.000000,0.000000,0.000000,0,0,0.000000,0.000000,NaN,NaN
2,10026,2011-03-01,J & J SNACK FOODS CORP,311821,7976,47.07000,14227.0,0.073175,9.0,0.283405,...,0.000000,0.000000,0.000000,0.000000,0,0,0.000000,0.000000,NaN,NaN
3,10026,2011-04-01,J & J SNACK FOODS CORP,311821,7976,50.82000,8807.0,0.079669,9.1,0.307888,...,0.000000,0.000000,0.000000,0.000000,0,0,0.000000,0.000000,NaN,NaN
4,10026,2011-05-01,J & J SNACK FOODS CORP,311821,7976,51.52000,9658.0,0.013774,9.0,0.346291,...,0.000000,0.000000,0.000000,0.000000,0,0,0.000000,0.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,10026,2020-09-01,J & J SNACK FOODS CORP,311813,7976,130.39000,30761.0,-0.036668,7.8,1.332004,...,0.054824,0.020844,0.000805,0.006202,0,0,0.191851,0.307767,0.347353,0.385106
117,10026,2020-10-01,J & J SNACK FOODS CORP,311813,7976,135.57001,26702.0,0.039727,6.8,1.130982,...,0.088271,0.024287,0.001009,0.007900,0,0,0.088231,0.212591,0.295336,0.352784
118,10026,2020-11-01,J & J SNACK FOODS CORP,311813,7976,145.39000,31460.0,0.072435,6.7,1.122155,...,0.076720,0.023280,0.000953,0.007071,0,0,0.107847,0.272321,0.342418,0.387752
119,10026,2020-12-01,J & J SNACK FOODS CORP,311813,7976,155.37000,22162.0,0.072598,6.7,1.323012,...,0.073992,0.021394,0.000937,0.006976,0,0,0.114898,0.281613,0.371416,0.425349
